<a href="https://colab.research.google.com/github/shivammehta007/NLPinEnglishLearning/blob/master/Sequence_2_sequence_Generation/Sequence2SequenceQuestionGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question Generation

Additional Dependencies

In [0]:
%%capture
!pip install fairseq

In [2]:
!rm -rf NLPinEnglishLearning
import os
from getpass import getpass
import urllib
import subprocess

def run_command(command):

    process = subprocess.Popen(
        command,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        shell=True,
        encoding="utf-8",
        errors="replace",
    )

    while True:
        realtime_output = process.stdout.readline()

        if realtime_output == "" and process.poll() is not None:
            break

        if realtime_output:
            print(realtime_output.strip(), flush=True)


cmd_string = 'git clone https://github.com/shivammehta007/NLPinEnglishLearning.git'
run_command(cmd_string)

Cloning into 'NLPinEnglishLearning'...
Checking out files:  94% (127/134)
Checking out files:  95% (128/134)
Checking out files:  96% (129/134)
Checking out files:  97% (130/134)
Checking out files:  98% (132/134)
Checking out files:  99% (133/134)
Checking out files: 100% (134/134)
Checking out files: 100% (134/134), done.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Download Glove from Kaggle

In [0]:
import os
import json

kaggle_info = json.load(open("/content/drive/My Drive/kaggle.json"))
os.environ['KAGGLE_USERNAME'] = kaggle_info["username"]
os.environ['KAGGLE_KEY'] = kaggle_info["key"]

In [5]:
!kaggle datasets list --user thanakomsn

ref                        title               size  lastUpdated          downloadCount  
-------------------------  -----------------  -----  -------------------  -------------  
thanakomsn/glove6b300dtxt  glove.6B.300d.txt  386MB  2017-11-28 07:19:43           2926  


In [16]:
!kaggle datasets download thanakomsn/glove6b300dtxt 

 97% 376M/386M [00:05<00:00, 80.7MB/s]
100% 386M/386M [00:06<00:00, 66.9MB/s]


In [0]:
%mkdir .vector_cache
%mv glove6b300dtxt.zip .vector_cache/

In [18]:
!unzip .vector_cache/glove6b300dtxt.zip
%ls -a .vector_cache/

Archive:  .vector_cache/glove6b300dtxt.zip
  inflating: glove.6B.300d.txt       
./  ../  glove6b300dtxt.zip


## Training Baseline Model

In [0]:
%cd QuestionGenerator/Sequence_2_sequence_Generation/Baseline

/content/QuestionGenerator/Sequence_2_sequence_Generation/Baseline


### Download Dataset

In [0]:
!python datadownloader.py --help

usage: datadownloader.py [-h] [-d DATASET] [-f]

Utility to download datasets currently available datasets: SQUAD

optional arguments:
  -h, --help            show this help message and exit
  -d DATASET, --dataset DATASET
                        Name of Dataset
  -f, --force           Skip Directory check and force override download


### PreProcess Dataset

In [0]:
!python preprocessing.py --help

usage: preprocessing.py [-h] [-d DATASET] [-m MODE] [-f]

Utility to Preprocess datasets currently available datasets: SQUAD

optional arguments:
  -h, --help            show this help message and exit
  -d DATASET, --dataset DATASET
                        Name of Dataset
  -m MODE, --mode MODE  Split on ANSWER or QUESTION
  -f, --filter          filter the sentences on answers


### Train Model

In [0]:
!python train.py --help

usage: train.py [-h] [-d DATASET] [-m MODEL] [-c CLIPNORM] [-l LEARNINGRATE]
                [-v] [-e EPOCHS] [-t TEACHERFORCING] [-tmp TRAINED_MODEL_PATH]

Utility to Train datasets {1: 'VanillaSeq2Seq'}

optional arguments:
  -h, --help            show this help message and exit
  -d DATASET, --dataset DATASET
                        which dataset to train on
  -m MODEL, --model MODEL
                        Which Model to Train
  -c CLIPNORM, --clipnorm CLIPNORM
                        Value to clip gradients
  -l LEARNINGRATE, --learningrate LEARNINGRATE
                        Learning rate of Adam Optmizer
  -v, --validation      Flag to turn validation on and off
  -e EPOCHS, --epochs EPOCHS
                        Number of Epochs to train
  -t TEACHERFORCING, --teacherforcing TEACHERFORCING
                        Teacher Forcing
  -tmp TRAINED_MODEL_PATH, --trained-model-path TRAINED_MODEL_PATH
                        Load the model from the directory


### Inference Model

In [0]:
!python inference.py --help

usage: inference.py [-h] [-d DATASET] [-m MODEL] [-ml MAX_LEN]
                    [-l MODEL_LOCATION]

Utility to generate Inference

optional arguments:
  -h, --help            show this help message and exit
  -d DATASET, --dataset DATASET
                        which dataset to train on
  -m MODEL, --model MODEL
                        Which Model to Train
  -ml MAX_LEN, --max-len MAX_LEN
                        Max length of the question to be generated
  -l MODEL_LOCATION, --model-location MODEL_LOCATION
                        Location of Model File


# Other Models

In [19]:
%cd ..

/content/NLPinEnglishLearning/Sequence_2_sequence_Generation/FairSeq_models/data


In [0]:
%cd ../FairSeq_models/

/content/QuestionGenerator/Sequence_2_sequence_Generation/FairSeq_models


In [0]:
!python preprocess.py

[INFO | preprocess.py:92 -           preprocess() ] Running FairSeq Preprocessing to convert files into fairseq binaries
Namespace(align_suffix=None, alignfile=None, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data/fairseq_binaries', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=1000, lr_scheduler='fixed', memory_efficient_fp16=False, min_loss_scale=0.0001, no_progress_bar=False, nwordssrc=45000, nwordstgt=28000, only_source=False, optimizer='nag', padding_factor=8, seed=1234, source_lang='sentence', srcdict=None, target_lang='question', task='translation', tensorboard_logdir='', testpref='data/processed/test', tgtdict=None, threshold_loss_scale=None, thresholdsrc=0, thresholdtgt=0, tokenizer=None, trainpref='data/processed/train', user_dir=None, validpref='data/processed/valid', workers=1)
| [sentence] Dictionary: 44999 types
| [sentence] da

In [0]:
!python train.py --help

usage: train.py [-h] [-m {LSTM,CNN,Transformer}] [-n NUM_EPOCHS]
                [-b BATCH_SIZE]

Utility to train the models

optional arguments:
  -h, --help            show this help message and exit
  -m {LSTM,CNN,Transformer}, --model {LSTM,CNN,Transformer}
                        Select the Seq2Seq Model to train
  -n NUM_EPOCHS, --num-epochs NUM_EPOCHS
                        Number of epochs to train
  -b BATCH_SIZE, --batch-size BATCH_SIZE
                        Training Batch Size


In [0]:
!python generate.py --help

usage: generate.py [-h] [-m {LSTM,CNN}] [-sm {best,last}] [-b BATCH_SIZE]

Utility to Generate Sentences from Test Set

optional arguments:
  -h, --help            show this help message and exit
  -m {LSTM,CNN}, --model {LSTM,CNN}
                        Select the Seq2Seq Model to train
  -sm {best,last}, --sub-model {best,last}
                        Select which model to generate with the one with best
                        valid loss or the last epoch trained model
  -b BATCH_SIZE, --batch-size BATCH_SIZE
                        Training Batch Size


# Evaluation

In [0]:
%%capture
!pip install git+https://github.com/Maluuba/nlg-eval.git@master

In [7]:
!nlg-eval --setup

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Installing to /root/.cache/nlgeval
In case of incomplete downloads, delete the directory and run `nlg-eval --setup /root/.cache/nlgeval' again.
glove2word2vec.py: 100% 1.00/1.00 [00:00<00:00, 467 chunks/s]
dictionary.txt: 550 chunks [00:01, 377 chunks/s]
uni_skip.npz: 100% 634/634 [00:40<00:00, 15.7 chunks/s]
uni_skip.npz.pkl: 100% 1.00/1.00 [00:00<00:00, 1.37k chunks/s]
bi_skip.npz: 100% 276/276 [00:18<00:00, 14.7 chunks/s]
bi_skip.npz.pkl: 100% 1.00/1.00 [00:00<00:00, 1.74k chunks/s]
multi-bleu.perl: 100% 1.00/1.00 [00:00<00:00, 370 chunks/s]
utable.npy: 100% 2.23k/2.23k [02:20<00:00, 15.9 chunks/s]
btable.npy: 100% 2.23k/2.23k [02:24<00:00, 15.5 chunks/s]
glove.6B.zip: 100% 823/823 [06:29<00:00, 2.11 chunks/s]
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for

In [6]:
%cd NLPinEnglishLearning/Sequence_2_sequence_Generation/FairSeq_models/data/raw/

/content/NLPinEnglishLearning/Sequence_2_sequence_Generation/FairSeq_models/data/raw


## LSTM

In [0]:
!grep ^H lstm.out | cut -f3- > lstm.out.sys
!grep ^T lstm.out | cut -f2- > lstm.out.ref

In [16]:
!nlg-eval --hypothesis=lstm.out.sys --references=lstm.out.ref

Using data from /root/.cache/nlgeval
In case of broken downloads, remove the directory and run setup again.
Bleu_1: 0.341780
Bleu_2: 0.185160
Bleu_3: 0.117828
Bleu_4: 0.079173
METEOR: 0.129704
ROUGE_L: 0.337291
CIDEr: 0.711782
WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.

Aborted!


## CNN

In [0]:
!grep ^H cnn.out | cut -f3- > cnn.out.sys
!grep ^T cnn.out | cut -f2- > cnn.out.ref

In [20]:
!nlg-eval --hypothesis=cnn.out.sys --references=cnn.out.ref

Using data from /root/.cache/nlgeval
In case of broken downloads, remove the directory and run setup again.
Bleu_1: 0.322365
Bleu_2: 0.166536
Bleu_3: 0.103490
Bleu_4: 0.067834
METEOR: 0.125735
ROUGE_L: 0.323530
CIDEr: 0.578102

Aborted!


## Tranformer

In [0]:
!grep ^H transformer.out | cut -f3- > transformer.out.sys
!grep ^T transformer.out | cut -f2- > transformer.out.ref

In [23]:
!nlg-eval --hypothesis=transformer.out.sys --references=transformer.out.ref

Using data from /root/.cache/nlgeval
In case of broken downloads, remove the directory and run setup again.
Bleu_1: 0.277407
Bleu_2: 0.113273
Bleu_3: 0.059634
Bleu_4: 0.034863
METEOR: 0.072429
ROUGE_L: 0.272074
CIDEr: 0.172780
WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
^C
